In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../data/train.csv')

In [ ]:
df.head()
print(df.shape)
df.info()

In [ ]:
all_columns = df.columns.values
# print(all_columns)
column_of_interest = ['SEXVAR', 'GENHLTH', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH', 'PRIMINSR', 'PERSDOC3', 'MEDCOST1', 'CHECKUP1', 'EXERANY2', 'SLEPTIM1', 'LASTDEN4', 'CVDSTRK3', 'ASTHMA3', 'ASTHNOW', 'CHCSCNC1', 'CHCOCNC1', 'CHCCOPD3', 'ADDEPEV3', 'CHCKDNY2', 'HAVARTH4', 'DIABETE4', 'WEIGHT2', 'HEIGHT3', 'DECIDE', 'DIFFWALK', 'DIFFDRES', 'DIFFALON', 'SMOKE100', 'SMOKDAY2', 'USENOW3', 'ECIGNOW2', 'LCSFIRST', 'LCSLAST', 'LCSNUMCG', 'ALCDAY4', 'AVEDRNK3', 'DRNK3GE5', 'MAXDRNKS', 'FLUSHOT7', 'HIVRISK5', 'COVIDPOS', 'COVIDSMP', 'COVIDPRM', 'PDIABTS1', 'PREDIAB2', 'DIABTYPE', 'INSULIN1', 'CHKHEMO3', 'EYEEXAM1', 'FEETSORE', 'TOLDCFS', 'HAVECFS', 'WORKCFS', 'COVIDVA1', 'SHINGLE2', 'COPDCOGH', 'COPDFLEM', 'COPDBRTH', 'COPDBTST', 'COPDSMOK', 'CNCRDIFF', 'CNCRAGE', 'CNCRTYP2', 'CSRVDOC1', 'CIMEMLOS', 'CDHOUSE', 'CDASSIST', 'CDHELP', 'CDSOCIAL', 'ACEDEPRS', 'ACEDRINK', 'ACEDRUGS', 'ACEPRISN', 'ACEPUNCH', 'LSATISFY', 'EMTSUPRT', 'SDHISOLT', 'SDHEMPLY', 'FOODSTMP', 'SDHFOOD1', 'SDHSTRE1', 'MARIJAN1', 'MARJSMOK']

columns_to_delete = ['_STATE', 'FMONTH', 'IDATE', 'IMONTH', 'IDAY', 'IYEAR', 'SEQNO', 'Num', 'PVTRESD1', 'COLGHOUS', 'STATERE1', 'CELPHON1', 'NUMADULT', 'RESPSLCT', 'SAFETIME', 'CELLFON5', 'PVTRESD3', 'CCLGHOUS', 'CSTATE1', 'LANDLINE', 'CHILDREN']

k = 0
for c in column_of_interest :
    if c not in all_columns :
        print(f"column '{c}' is not in dataframe")
    else :
        k+=1
        #print("good")
        pass
print(k)


for c in columns_to_delete :
    if c not in all_columns :
        print(f"column '{c}' is not in dataframe")
    else :
        #print("good")
        pass



# Feature engineering

In [ ]:
"""
Heatmap visualization using Plotly.
Colors will be:
    from -inf to central_limit_val : black
    from central_limit_val to +inf : blue to pink shades
Args:
    matrix: 2D array containing the data values
    xlabels: Labels for x-axis (columns)
    ylabels: Labels for y-axis (rows)
    central_limit_val: Define color thresold. If None, auto-calculated
    plot: Whether to display the plot
    full_plotly_offline: Whether to include full Plotly JS in HTML output
Returns:
    HTML string of the plot
"""

def plot_nan_proportion_heatmap(matrix, xlabels, ylabels, width=800, height=500, plot=True, full_plotly_offline=False):
    # Convert to DataFrame
    df = pd.DataFrame(matrix, columns=xlabels, index=ylabels)

    # Create a NaN indicator matrix: 1 for NaN, 0 for non-NaN
    nan_indicator = df.isna().astype(int)

    # Plot the NaN indicator matrix
    fig = px.imshow(
        nan_indicator,
        width=width,
        height=height,
        template="plotly_white",
        color_continuous_scale="Greys",  # Use a grayscale gradient
        labels={"color": "NaN Proportion"},
        aspect="auto",
        zmin=0,
        zmax=1,
    )
    fig.update_xaxes(side="top", tickangle=45, tickfont=dict(size=5))
    fig.update_layout(
        margin=dict(pad=1),
        # title="Proportion of NaN Values (1 = NaN, 0 = Non-NaN)",

    )

    # Return or show
    if plot:
        fig.show()
    if full_plotly_offline:
        return fig.to_html(full_html=True)
    else:
        return fig.to_html(full_html=False, include_plotlyjs=False)


plot_nan_proportion_heatmap(df.head(100), xlabels=column_of_interest, ylabels=range(100))


In [ ]:
def calculate_nan_proportion(matrix, xlabels):
    """
    Calculate the proportion of NaN values in each column.

    Args:
        matrix: 2D array or DataFrame containing the data.
        xlabels: Labels for the columns.

    Returns:
        A DataFrame with columns and their NaN proportions.
    """
    df = pd.DataFrame(matrix, columns=xlabels)
    nan_proportion = df.isna().mean(axis=0)  # Proportion of NaNs per column
    return nan_proportion

nans_prop = calculate_nan_proportion(df, xlabels=column_of_interest)

plt.bar(column_of_interest, height=nans_prop)
print(nans_prop)


In [ ]:
nan_threshold = 0.4

print(f"We remove columns containing more than {nan_threshold*100} % of NaNs")
rem_cols = nans_prop[nans_prop > nan_threshold]
print(rem_cols)
print(f"Number of columns to remove {len(rem_cols)}")

df = df[column_of_interest]
df = df[~df.isin(rem_cols)]

df.head()

Instead of removing all lines with NaNs w

In [ ]:
acp_df = PCA(n_components=10)
acp_df.fit(df)